<h3 style="text-align: center;"><strong>ESCUELA DE INGENIERÍA INFORMÁTICA - UNIVERSIDAD DE SEVILLA</strong></h3>
<h3 style="text-align: center;"><strong>GRADO EN INGENIERÍA INFORMÁTICA</strong></h3>
<h1 style="font-size: 1em; text-align: center;"><strong>MATEMÁTICA DISCRETA<br /></strong></h1>
<h2 style="font-size: 1.17em; text-align: center;"><strong>Práctica 6. Estrategias ganadoras en juegos de estrategia a tiempo real.<br /></strong></h2>
<p>En este documento práctico tomaremos como referencia el juego de estrategia <strong>Stellaris</strong>, de la empresa Paradox Interactive, con el fin de emplear el estudio de grafos en esta categoría de juegos para ejecutar posibles estrategias que nos ofrezca ventajas o incluso nos garantice la victoria.</p>
<p>Esta práctica la dividiremos en los siguientes apartados:</p>
<p> 1. <a href= "#Grafo">Representación del mapa</a>.</p>
<p> 2. <a href= "#Exploracion">¿Cómo saber si se tiene un buen comienzo y cómo ejecutar la exploración?</a></p>
<p> 3. <a href= "#Ubicacion">Ubicaciones de la capital</a>.</p>
<p> 4. <a href= "#Comercio">Estudio del comercio</a>.</p>
<p> 5. <a href= "#Conquista">Conquista eficiente</a>.</p>
<p> 6. <a href= "#Defensa">Defensa del imperio</a>.</p>
<p> </p>
<p>Al final de la práctica hay una  <a href="#cuestiones">colección de otros ejercicios propuestos.</a></p>
<p> </p>
<p>Antes de empezar con la práctica, hagamos una pequeña introducción a Stellaris para así poder entender en qué consiste el videojuego y tener contexto para poder tener una mejor toma de decisiones a la hora de plantear una estrategia:</p>
<p><strong>Stellaris</strong> es un juego de gran estrategia en tiempo real ambientado en el espacio, cuyos combates se centran más en su preparación y estrategia que en el propio combate. También hay opciones diplomáticas, tales como alianzas y acuerdos comerciales con otras razas. El jugador comienza con un solo sistema en su poder. Cualquier partida empieza con la exploración y la colonización del espacio, para más avanzada la partida sus actividades se centren en la diplomacia y el cómo gobernar un imperio.</p>
<p>En este caso, nos centraremos en el aspecto de la exploración y la conquista de sistemas.</p>

<h2><a name= "Grafo"></a><strong>1. Representacion del mapa</strong></h2>
<p>Como se ha mencionado antes, al comenzar una partida tendremos en nuestro poder un sistema seleccionado de manera aleatoria y tendremos que explorar (y a su vez conquistar) los sistemas <strong>adyacentes</strong> al nuestro para así ir expandiendo nuestro territorio.</p>
<p>El problema que tenemos es que no sabemos de forma gráfica cómo es la galaxia en la que nos encontramos, o dicho de la mejor forma, no sabemos cómo es el <strong>mapa</strong> de la partida, ya que éste varía completamente cuando empezamos otras partidas.</p>
<p>Para solucionar este problema haremos una <strong>representación gráfica</strong> del mapa de nuestra partida.</p>
<p>La función <em>(nombre de la función)</em> nos representará el mapa de nuestra partida como un grafo. Cada partida estará nombrada como <em>"gamestate_.txt"</em>, donde '_' es un <strong>número del 0 al 9</strong>.</p>

<p>Como se puede observar (y como podrás suponer), el grafo que devuelve la función <em>(nombre de la función)</em> es el mapa de la galaxia de tu partida, sin filtros. A simple vista el grafo no dice nada, simplemente proyecta los <strong>sistemas de la galaxia</strong> (vértices) y las <strong>conexiones que existen entre estas</strong> (aristas).</p>
<p>La siguiente sucesión de funciones nos representará el mapa con información extra:</p>
<p>1) (Inserte info de la representación)</p>

<p>2) (Inserte info de la representación)</p>

<p>3) (Inserte info de la representación)</p>

<h2><a name="Exploracion"></a><strong>2. ¿Cómo saber si se tiene un buen comienzo y cómo ejecutar la exploración?</strong></h2>
<p>Sabiendo ya la forma que tiene nuestra galaxia y cómo funciona, podremos ubicarnos en ésta y plantear ya nuestra estrategia para comenzar con nuestra expansión por el espacio.</p>
<p>Sólo hay una variable que hace que a la hora de tomar ciertas decisiones hagamos una cosa u otra. Así es, hablamos del <strong>comienzo</strong>.</p>
<p>Que hayamos empezado de <em>x</em> manera en la galaxia implica en nuestra manera de explorar y reunir información suficiente para conquistar o no ciertos sistemas y así extender nuestros dominios de la <strong>manera más eficiente</strong>. Una mala decisión puede acabar en una posible caída de nuestro imperio por posición estratégica.</p>
<br>...</br>

<h2><a name="Ubicacion"></a><strong>3. Ubicaciones de la capital</strong></h2>
<p>Ahora que conocemos nuestra galaxia y nos hemos podido abrir paso extendiendo nuestros dominios, llega un punto en el que debemos ubicar aquel sistema que será la <strong>capital</strong> de nuestro <strong>futuro imperio</strong>. El problema se basa en qué sistema o por qué lugar nos vendría bien situar nuestra capital, que esté lo más centrado posible para poder llegar rápidamente a cualquier lugar conquistado o incluso para poder defenderlo fácilemente de cualquier ataque enemigo.</p>
<p>Nos centraremos más en encontrar los sistemas perfectos para asignarles nuestra capital.</p>
<p>Mediante la función <em>(nombre de la función)</em> podremos ver los mejores lugares del imperio para colocar una capital.</p>